# Homework

This homework asks you to compute a function's derivatives using symbolic and automatic differentiation methods. Our objective function is the follows.

\begin{align}
 f(x) = \ln(x)\times \exp\left[-\frac{1}{2}\left(\frac{x-\mu}{\sigma}\right)^2 \right],
\end{align}
where $\ln(x)$ is the natural logarithm of $x$. (Note: In the majority of programming languages including Julia, the natural logarithm of $x$ is denoted as `log(x)`. If you wish to specify the logarithm of base 10, you use `log10(x)`.)

Let's assume $\mu=1.5$ and $\sigma=2$. 



---
## Automatic Differentiations

### Use the Taylor expansion to derive the result of logarithms on dual numbers.

Let $f(x) = \log(x)$ and so $f'(x) = \frac 1x$. For a dual number, it means $f(a+b\epsilon) = \log ({a+b\epsilon})$. Applying the Taylor expansion, we have

\begin{align}
 f(a +b\epsilon) & = f(a) + f'(a)b\epsilon = \log a + \frac 1a b\epsilon; \\
\Longrightarrow \quad \log ({a+b\epsilon}) & = \log a + \frac 1a b\epsilon.
\end{align}

---

### Add methods of the division ("/") and logarithm ("log()") for dual numbers in Julia.

We first define the *DualNumber* according to lecture notes

In [17]:
import Base: +, *, -, ^, exp, log

struct DualNumber{T1, T2} <: Real   
    re::T1                # differnt types so (Float64, Int64), etc., is possible
    du::T2
end

+(x::DualNumber, y::DualNumber) = DualNumber(x.re + y.re, x.du + y.du)  # dual addition
+(x::DualNumber, a::Number) = DualNumber(x.re + a, x.du)  
+(a::Number, x::DualNumber) = DualNumber(x.re + a, x.du) 

-(x::DualNumber, y::DualNumber) = DualNumber(x.re - y.re, x.du - y.du)  # dual subtraction
-(x::DualNumber, a::Number) = DualNumber(x.re - a, x.du)  
-(a::Number, x::DualNumber) = DualNumber(a - x.re , -x.du)  

*(x::DualNumber, y::DualNumber) = DualNumber(x.re*y.re, x.re*y.du + y.re*x.du)
*(x::DualNumber, a::Number) = DualNumber(a*x.re, a*x.du)
*(a::Number, x::DualNumber) = DualNumber(a*x.re, a*x.du)

function exp(x::DualNumber)
    exp_re = exp(x.re)
    return DualNumber(exp_re, x.du * exp_re)
end

# Where is the division rule? Your turn!

exp (generic function with 17 methods)

next, we define the division operator and logarithm

In [16]:
import Base: log, /


# define division operator
/(x::DualNumber, y::DualNumber) = DualNumber(x.re / y.re, (x.du * y.re - x.re * y.du) / (y.re^2))
/(x::DualNumber, a::Number) = DualNumber(x.re / a, x.du / a)
/(a::Number, x::DualNumber) = DualNumber(a / x.re, -a * x.du / (x.re^2))

# define logarithm
function log(x::DualNumber)
    if x.re <= 0
        throw(DomainError(x, "Logarithm undefined for non-positive reals as real part."))
    end
    return DualNumber(log(x.re), x.du / x.re)
end

log (generic function with 30 methods)

---

### Use the dual numbers you've defined to compute the derivative of $f(x)$ at $x=1.2$.

In [21]:
using Base: exp, log

# parameter setting
μ = 1.5
σ = 2.0
x0 = DualNumber(1.2, 1)  # x0 = 1.2 + ε, where ε is the infinitesimal part

# compute f(x0)
f_x0 = log(x0) * exp(-0.5 * ((x0 - μ) / σ)^2) 

# the real part is the function value at x0, while the dual part is the derivative value at x0
println("The value of f(x) at x = 1.2 is $(f_x0.re)")
println("The derivative of f(x) at x = 1.2 is $(f_x0.du)")

The value of f(x) at x = 1.2 is 0.1802819336716901
The derivative of f(x) at x = 1.2 is 0.8375320155347377


---

### Use Julia's package `ForwardDiff`, which implements the forward-mode auto differentiation, to compute the derivative of $f(x)$ at $x=1.2$.

In [28]:
using ForwardDiff

# define function f(x) as fun(x)
function fun(x)
    μ = 1.5
    σ = 2.0
    return log(x) * exp(-0.5 * ((x - μ) / σ)^2)
end

# compute derivative at x0 = 1.2
x0 = 1.2
dfun = ForwardDiff.derivative(fun, x0)

println("The derivative of f(x) at x = 1.2 is $dfun")


The derivative of f(x) at x = 1.2 is 0.8375320155347377


---

## Symbolic Differentiation

### Use Julia's package `SymPy` to compute the analytic derivative of $f(x)$ and evaluate the result at $x=1.2$.
- Hint: You may need to use `diff` and `subs` functions in the package.
- Remark: I intended to use `Symbolics` but find it less straightforward. You might give it a try.

In [31]:
using SymPy

# define symbols and function f(x)
x, μ, σ = symbols("x μ σ")
f = log(x) * exp(-1/2 * ((x - μ)/σ)^2)

# derivative for f(x)
df = diff(f, x)

# show the form of f(x) and derivative of f(x)
@show df
@show df.subs(Dict(x => 1.2, μ => 1.5, σ => 2))


df = -0.5*(2*x - 2*μ)*exp(-0.5*(x - μ)^2/σ^2)*log(x)/σ^2 + exp(-0.5*(x - μ)^2/σ^2)/x
df.subs(Dict(x => 1.2, μ => 1.5, σ => 2)) = 0.837532015534738


0.837532015534738

---